In [1]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.model_selection import TimeSeriesSplit
# from sklearn.metrics import mean_squared_error as mse

TEAMS = ['Burnley',
 'AFC Bournemouth',
 'Crystal Palace',
 'West Bromwich Albion',
 'Huddersfield Town',
 'Arsenal',
 'Liverpool',
 'Brighton & Hove Albion',
 'Manchester United',
 'Watford',
 'Newcastle United',
 'Chelsea',
 'Southampton',
 'Manchester City',
 'Swansea City',
 'Stoke City',
 'Tottenham Hotspur',
 'Leicester City',
 'West Ham United',
 'Everton']

# Extracts the names of the teams and the scores they got. 
def extract_match_results(matches):
    def f(row):
        # print(row)
        teams, scores = row['label'].strip().split(',')
        t1, t2 = teams.strip().split(' - ')
        s1, s2 = map(int, scores.strip().split(' - '))
        date = row['dateutc']
        return {'t1':t1, 't2':t2, 's1':s1, 's2':s2, 'date':date}
        # return {'s1': s1, 's2': s2, 'date': date, t1: 1, t2: -1}

    results = matches.apply(f, axis=1)
    # results = pd.DataFrame(results)

    results = pd.DataFrame(list(results))
    results = results.sort_values(by='date', axis=0).reset_index()
    # results = results.fillna(0)

    return results

# formats the results into a trainable table
def format_match_predict_score(match_results):
    arr1 = [1] + 

    
def train_expected_number_of_goals(prior_matches):
    results = extract_match_results(prior_matches)
    teamnames = sorted([team for team in results if team not in ['s1', 's2', 'date', 'index']])

    # doubling data by expressing it in terms of either team, with an extra column to track who is home
    results['home'] = 1
    y, X = results['s1'].values, results[['home'] + teamnames].values
    y2, X2 = results['s2'].values, -results[['home'] + teamnames].values
    y = np.concatenate([y, y2], axis=0)
    X = np.concatenate([X, X2], axis=0)

    lr: LinearRegression = LinearRegression().fit(X, y)



    return lambda match: lr.predict(match), lr



def train_next_goal_home_prior_model(prior_matches):

    expected_goals_predictor = train_expected_number_of_goals(prior_matches)

    def format_match(match):
        home, away = extract_teams_from_match(match)
        return np.array([1] + [(team == home) - (team == away) for team in teamnames]).reshape((1, -1))

    def predictor(match):
        features_home = format_match(match)
        print(features_home)
        expected_goals_home = lr.predict(features_home)
        features_away = -format_match(match)
        expected_goals_away = lr.predict(features_away)

        return expected_goals_home / (expected_goals_home + expected_goals_away)
    
    return predictor, expected_goals_predictor

In [ ]:
def test():
    matches_england = pd.read_json("../data/figshare/matches_England.json")



test()